In [1]:
# general 
import os

# data request
import requests
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

# data processing
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
style.use('seaborn')

# import functions
from utils import *

# Data Retrieval

## Getting all NBA teams from API endpoint

In [2]:
nba_teams = pd.DataFrame(teams.get_teams())
nba_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [3]:
team_ids = nba_teams['id'].unique()
team_ids

array([1610612737, 1610612738, 1610612739, 1610612740, 1610612741,
       1610612742, 1610612743, 1610612744, 1610612745, 1610612746,
       1610612747, 1610612748, 1610612749, 1610612750, 1610612751,
       1610612752, 1610612753, 1610612754, 1610612755, 1610612756,
       1610612757, 1610612758, 1610612759, 1610612760, 1610612761,
       1610612762, 1610612763, 1610612764, 1610612765, 1610612766])

## Getting all games from season 2014-2015 onwards
<br>

There should be 30 x 82 = 2460 games record for each season. 
- when Team A play against Team B, there will be 2 records, A vs B and B vs A
<br><br>

We first set the start and end date for each season, then we retrieve games played between the respective period for each season. At the end, we will do a count check for the number of records in each season. 

In [30]:
#2014-15 season dates
start_2014 = '2014-10-28'
end_2014 = '2015-04-15'

#2015-16 season dates
start_2015 = '2015-10-27'
end_2015 = '2016-04-13'

#2016-17 season dates
start_2016 = '2016-10-25'
end_2016 = '2017-04-12'

#2017-18 season dates
start_2017 = '2017-10-17'
end_2017 = '2018-04-11'

#2018-19 season dates
start_2018 = '2018-10-16'
end_2018 = '2019-04-10'

#2019-20 season dates
start_2019 = '2019-10-22'
end_2019 = '2020-03-11' # ends earlier cos after that we head into the bubble

#2020-21 season dates
start_2020 = '2020-12-22' # current on-going season
end_2020 = '2021-05-16'

In [32]:
all_games = pd.DataFrame()
all_team_df = []
for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    all_team_df.append(games)
    games = games[
        (games['GAME_DATE'].between(start_2014, end_2014)) | 
        (games['GAME_DATE'].between(start_2015, end_2015)) | 
        (games['GAME_DATE'].between(start_2016, end_2016)) | 
        (games['GAME_DATE'].between(start_2017, end_2017)) | 
        (games['GAME_DATE'].between(start_2018, end_2018)) | 
        (games['GAME_DATE'].between(start_2019, end_2019)) | 
        (games['GAME_DATE'].between(start_2020, end_2020))
    ]
    all_games = all_games.append(games)

all_games = all_games.sort_values('GAME_DATE', ascending=False)

all_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
10,22020,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16,ATL vs. HOU,W,240,124,...,0.731,11.0,45.0,56.0,32,10.0,8,13,16,29.0
0,22020,1610612753,ORL,Orlando Magic,0022001075,2021-05-16,ORL @ PHI,L,240,117,...,0.854,9.0,38.0,47.0,23,4.0,3,16,25,-11.0
0,22020,1610612740,NOP,New Orleans Pelicans,0022001072,2021-05-16,NOP vs. LAL,L,241,98,...,0.714,8.0,39.0,47.0,26,4.0,5,21,18,-12.0
0,22020,1610612741,CHI,Chicago Bulls,0022001068,2021-05-16,CHI vs. MIL,W,241,118,...,0.737,8.0,37.0,45.0,23,8.0,8,16,20,6.0
7,22020,1610612742,DAL,Dallas Mavericks,0022001071,2021-05-16,DAL @ MIN,L,239,121,...,0.850,11.0,27.0,38.0,32,5.0,5,16,19,-15.0


In [33]:
# checking if the number of games record for each season is correct
all_games.groupby(all_games.SEASON_ID.str[-4:])[['GAME_ID']].count()

,GAME_ID
SEASON_ID,
2014,2460
2015,2460
2016,2460
2017,2460
2018,2460
2019,1942
2020,2160


## Data Cleaning
- what features are there?
- any null values?
- converting game date to datetime format
- splitting games records into seasons

In [10]:
all_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16310 entries, 0 to 642
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          16310 non-null  object 
 1   TEAM_ID            16310 non-null  int64  
 2   TEAM_ABBREVIATION  16310 non-null  object 
 3   TEAM_NAME          16310 non-null  object 
 4   GAME_ID            16310 non-null  object 
 5   GAME_DATE          16310 non-null  object 
 6   MATCHUP            16310 non-null  object 
 7   WL                 16310 non-null  object 
 8   MIN                16310 non-null  int64  
 9   PTS                16310 non-null  int64  
 10  FGM                16310 non-null  int64  
 11  FGA                16310 non-null  int64  
 12  FG_PCT             16310 non-null  float64
 13  FG3M               16310 non-null  int64  
 14  FG3A               16310 non-null  float64
 15  FG3_PCT            16310 non-null  float64
 16  FTM                16310

In [46]:
all_games = pd.read_csv('../data/annual_data/annual_nba_data.csv')

all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])
all_games = all_games.sort_values(by = ["GAME_DATE", "GAME_ID"], ascending = True)
all_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
16396,22014,1610612740,NOP,New Orleans Pelicans,21400001,2014-10-28,NOP vs. ORL,W,240,101,...,0.484,26.0,36.0,62.0,20,10.0,17,9,17,17.0
16398,22014,1610612753,ORL,Orlando Magic,21400001,2014-10-28,ORL @ NOP,L,239,84,...,0.762,16.0,40.0,56.0,17,5.0,9,18,25,-17.0
16400,22014,1610612759,SAS,San Antonio Spurs,21400002,2014-10-28,SAS vs. DAL,W,241,101,...,0.813,9.0,29.0,38.0,23,5.0,3,20,20,1.0
16401,22014,1610612742,DAL,Dallas Mavericks,21400002,2014-10-28,DAL @ SAS,L,239,100,...,0.842,9.0,24.0,33.0,17,9.0,3,10,20,-1.0
16397,22014,1610612747,LAL,Los Angeles Lakers,21400003,2014-10-28,LAL vs. HOU,L,240,90,...,0.795,11.0,25.0,36.0,16,7.0,3,11,32,-18.0


### Output df to csv file

In [47]:
all_games.to_csv('../data/annual_data/annual_nba_data.csv', index=False)

# Data Processing

In [48]:
all_games = pd.read_csv("../data/annual_data/annual_nba_data.csv")

In [49]:
nba_teams = get_nba_teams()

Retrieving NBA teams...
Succesfully retrieved NBA teams.


## Spliting the data by seasons and calculations additonal team stats
- split by season
- create a dictionary to store the latest team statistics
- replace WL to numeric number
- merging 2 data records of the same game id into 1 row. A vs B and B vs A with same game id are merged
- calculate new stats for each team after every game
- disparity of stats between teams will be calculated before the calculating teams' latest stats after their latest games. Disparity is always calculated taking team_x - team_y (based on columns header)
- rearrange columns 
- output each season data into its own respective csv

In [50]:
# splitting data into different seasons
season_ids = all_games['SEASON_ID'].unique()
for sid in season_ids:
    team_stats = populate_team_stats(nba_teams)

    season = all_games[all_games['SEASON_ID'] == sid]
    season.reset_index(drop=True, inplace=True)
    
    # change W to 1, L to 0
    season = season.replace(['W','L'], [1, 0])

    # merge 2 rows of the same game id together, A vs B and B vs A with same game id are merged
    season_merged = season.merge(season, on='GAME_ID')
    season_merged = season_merged.drop(season_merged[season_merged['TEAM_ID_x'] == season_merged['TEAM_ID_y']].index)
    season_merged = season_merged.drop(season_merged[season_merged['MATCHUP_x'].str.contains("@")].index)
    # season_merged.drop_duplicates(subset=['GAME_ID'], inplace=True)

    season_merged[['GAME_NO_x', 'GAME_NO_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y']].apply(lambda x: update_GAME_NO(team_stats, *x), axis=1, result_type='expand')
    season_merged[['HOME_COURT_x', 'HOME_COURT_y']] = season_merged[['MATCHUP_x']].apply(lambda x: update_HOME_COURT(*x), axis=1, result_type='expand')
    season_merged[['cPTS_x', 'cPTS_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'PTS_x', 'PTS_y']].apply(lambda x: update_cPTS(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cPTS_ALLOWED_x', 'cPTS_ALLOWED_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'PTS_y', 'PTS_x']].apply(lambda x: update_cPTS_ALLOWED(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cAST_x', 'cAST_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'AST_x', 'AST_y']].apply(lambda x: update_cAST(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cOREB_x', 'cOREB_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'OREB_x', 'OREB_y']].apply(lambda x: update_cOREB(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cDREB_x', 'cDREB_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'DREB_x', 'DREB_y']].apply(lambda x: update_cDREB(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cFGM_x', 'cFGM_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'FGM_x', 'FGM_y']].apply(lambda x: update_cFGM(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cFG3M_x', 'cFG3M_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'FG3M_x', 'FG3M_y']].apply(lambda x: update_cFG3M(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cFGA_x', 'cFGA_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'FGA_x', 'FGA_y']].apply(lambda x: update_cFGA(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cTO_x', 'cTO_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'TOV_x', 'TOV_y']].apply(lambda x: update_cTO(team_stats, *x), axis=1, result_type='expand')
    season_merged[['cFTA_x', 'cFTA_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'FTA_x', 'FTA_y']].apply(lambda x: update_cFTA(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_PTS', 'AVG_PTS_x', 'AVG_PTS_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y','GAME_NO_x', 'GAME_NO_y', 'cPTS_x', 'cPTS_y']].apply(lambda x: update_AVG_PTS(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_AST', 'AVG_AST_x', 'AVG_AST_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y','GAME_NO_x', 'GAME_NO_y', 'cAST_x', 'cAST_y']].apply(lambda x: update_AVG_AST(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_OREB', 'AVG_OREB_x', 'AVG_OREB_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y','GAME_NO_x', 'GAME_NO_y', 'cOREB_x', 'cOREB_y']].apply(lambda x: update_AVG_OREB(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_DREB', 'AVG_DREB_x', 'AVG_DREB_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y','GAME_NO_x', 'GAME_NO_y', 'cDREB_x', 'cDREB_y']].apply(lambda x: update_AVG_DREB(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_OFF_EFF', 'OFF_EFF_x', 'OFF_EFF_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'cPTS_x', 'cPTS_y', 'cFGA_x', 'cFGA_y', 'cOREB_x', 'cOREB_y', 'cTO_x', 'cTO_y', 'cFTA_x', 'cFTA_y']].apply(lambda x: update_OFF_EFF(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_DEF_EFF', 'DEF_EFF_x', 'DEF_EFF_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'cPTS_ALLOWED_x', 'cPTS_ALLOWED_y', 'cFGA_x', 'cFGA_y', 'cOREB_x', 'cOREB_y', 'cTO_x', 'cTO_y', 'cFTA_x', 'cFTA_y']].apply(lambda x: update_DEF_EFF(team_stats, *x), axis=1, result_type='expand')
    season_merged[['EFG_x', 'EFG_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'cFGM_x', 'cFGM_y', 'cFG3M_x', 'cFG3M_y', 'cFGA_x', 'cFGA_y']].apply(lambda x: update_EFG(team_stats, *x), axis=1, result_type='expand')
    season_merged[['DIS_ELO', 'ELO_x', 'ELO_y']] = season_merged[['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y', 'WL_x']].apply(lambda x: update_ELO(team_stats, *x), axis=1, result_type='expand')

    season_final_df = season_merged[['SEASON_ID_x', 'TEAM_ID_x', 'TEAM_ABBREVIATION_x', 'TEAM_NAME_x','GAME_ID', 'GAME_DATE_x', 'MATCHUP_x', 'WL_x', 'MIN_x', 'PTS_x','FGM_x', 'FGA_x', 'FG_PCT_x', 'FG3M_x', 'FG3A_x', 'FG3_PCT_x', 'FTM_x','FTA_x', 'FT_PCT_x', 'OREB_x', 'DREB_x', 'REB_x', 'AST_x', 'STL_x','BLK_x', 'TOV_x', 'PF_x', 'PLUS_MINUS_x', 'GAME_NO_x', 'HOME_COURT_x', 'cPTS_x', 'cPTS_ALLOWED_x', 'cAST_x', 'cOREB_x', 'cDREB_x', 'cFGM_x', 'cFG3M_x', 'cFGA_x','cTO_x', 'cFTA_x', 'AVG_PTS_x','AVG_AST_x', 'AVG_OREB_x', 'AVG_DREB_x', 'OFF_EFF_x', 'DEF_EFF_x','EFG_x', 'ELO_x', 'SEASON_ID_y', 'TEAM_ID_y','TEAM_ABBREVIATION_y', 'TEAM_NAME_y', 'GAME_DATE_y', 'MATCHUP_y','WL_y', 'MIN_y', 'PTS_y', 'FGM_y', 'FGA_y', 'FG_PCT_y', 'FG3M_y','FG3A_y', 'FG3_PCT_y', 'FTM_y', 'FTA_y', 'FT_PCT_y', 'OREB_y', 'DREB_y','REB_y', 'AST_y', 'STL_y', 'BLK_y', 'TOV_y', 'PF_y', 'PLUS_MINUS_y', 'GAME_NO_y', 'HOME_COURT_y', 'cPTS_y','cPTS_ALLOWED_y', 'cAST_y', 'cOREB_y','cDREB_y', 'cFGM_y', 'cFG3M_y', 'cFGA_y', 'cTO_y', 'cFTA_y', 'AVG_PTS_y', 'AVG_AST_y', 'AVG_OREB_y','AVG_DREB_y', 'OFF_EFF_y', 'DEF_EFF_y', 'EFG_y', 'ELO_y', 'DIS_PTS', 'DIS_AST', 'DIS_OREB', 'DIS_DREB', 'DIS_OFF_EFF', 'DIS_DEF_EFF', 'DIS_ELO']]

    season_final_df.to_csv(f"../data/annual_data/season_{sid}_data.csv", index=False)

Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.
Populating team stats...
Successfully populated team stats.


In [42]:
s2014 = pd.read_csv('../data/annual_data/season_22014_data.csv')
s2015 = pd.read_csv('../data/annual_data/season_22015_data.csv')
s2016 = pd.read_csv('../data/annual_data/season_22016_data.csv')
s2017 = pd.read_csv('../data/annual_data/season_22017_data.csv')
s2018 = pd.read_csv('../data/annual_data/season_22018_data.csv')
s2019 = pd.read_csv('../data/annual_data/season_22019_data.csv')
s2020 = pd.read_csv('../data/annual_data/season_22020_data.csv')

In [43]:
x = pd.concat([s2014, s2015, s2016, s2017, s2018, s2019, s2020], ignore_index=True)

In [45]:
x.to_csv(f"../data/annual_data/annual_data_processed.csv", index=False)